In [ ]:
import deepdih
from rdkit import Chem

In [ ]:
rdmol = Chem.MolFromMolFile("ligand.mol", removeHs=False, sanitize=True)
fragments = deepdih.mollib.get_fragments(rdmol)
for n_frag, frag in enumerate(fragments):
    deepdih.utils.write_sdf(frag, f"fragment_{n_frag}.mol")

In [ ]:
# optimize the fragments using GFN2-xTB
from tblite.ase import TBLite

opt_calculator = TBLite(method="GFN2-xTB")

for n_frag, frag in enumerate(fragments):
    rotamers = deepdih.utils.get_rotamers(frag)
    dih_results = []
    for rot in rotamers:
        dih_result_rot = deepdih.geomopt.dihedral_scan(frag, opt_calculator, rot, 12)
        dih_results.extend(dih_result_rot)
    deepdih.utils.write_sdf(dih_results, f"fragment_{n_frag}_dihedral_scan.sdf")

In [ ]:
# recalc single-point energy
from ase.calculators.psi4 import Psi4
from deepdih.geomopt import recalc_energy

for n_frag in range(len(fragments)):
    print(f"Fragment {n_frag}")
    conformations = deepdih.utils.read_sdf(f"fragment_{n_frag}_dihedral_scan.sdf")
    charge = Chem.GetFormalCharge(conformations[0])
    atoms = deepdih.utils.rdmol2atoms(conformations[0])
    calculator = Psi4(atoms = atoms,
        method = 'wb97x-d',
        memory = '8000MB',
        basis = 'def2-svp',
        num_threads = 8,
        charge=int(charge),
        multiplicity=1)
    recalc_confs = [deepdih.geomopt.recalc_energy(c, calculator) for c in conformations]
    deepdih.utils.write_sdf(recalc_confs, f"fragment_{n_frag}_dihedral_scan_recalc.sdf")

In [ ]:
atom = conformations[0].GetAtoms()[0]

In [ ]:
atom.GetAtomicNum()